<a href="https://colab.research.google.com/github/PC-11-00/Keyword-Searching/blob/main/Dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import numpy as np

In [2]:
with open('/content/DB.json', 'r') as file:
    # Load the JSON data
    data = json.load(file)



In [3]:
words = []
descriptions = []

In [4]:
for i in range(len(data)):
  words.append(data[i]["word"].lower())
  descriptions.append(data[i]["description"])


In [5]:
# Function to calculate Levenshtein distance between two words
def levenshtein_distance(word1, word2):
    m, n = len(word1), len(word2)
    dp = np.zeros((m+1, n+1))

    for i in range(m+1):
        dp[i][0] = i
    for j in range(n+1):
        dp[0][j] = j

    for i in range(1, m+1):
        for j in range(1, n+1):
            cost = 0 if word1[i-1] == word2[j-1] else 1
            dp[i][j] = min(dp[i-1][j] + 1, dp[i][j-1] + 1, dp[i-1][j-1] + cost)

    return dp[m][n]

In [18]:
import heapq

class TrieNode:
    def __init__(self):
        self.children = {}  # Mapping from character to child node
        self.original_words = set()  # Set to store original words

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, word):
        for i in range(len(word)):
            self._insert_suffix(word[i:], word)

    def _insert_suffix(self, suffix, original_word):
        node = self.root
        for char in suffix:
            if char not in node.children:
                node.children[char] = TrieNode()
            node = node.children[char]
            node.original_words.add(original_word)

    def search(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                return False
            node = node.children[char]
        return True

    def get_original_words(self, suffix):
        node = self.root
        for char in suffix:
            if char not in node.children:
                return None
            node = node.children[char]
        return node.original_words

    def get_closest_word(self, word):
        min_distance = float('inf')
        closest_word = None

        # Find all words in the Trie with similar prefix
        similar_prefix_words = self._find_similar_prefix_words(word)

        if similar_prefix_words:
            for original_word in similar_prefix_words:
                distance = self._calculate_edit_distance(word, original_word)
                if distance < min_distance:
                    min_distance = distance
                    closest_word = original_word

        return closest_word

    def _find_similar_prefix_words(self, word):
        node = self.root
        similar_prefix_words = set()
        for char in word:
            if char not in node.children:
                return similar_prefix_words
            node = node.children[char]
            similar_prefix_words.update(node.original_words)
        return similar_prefix_words

    def _calculate_edit_distance(self, word1, word2):
        m, n = len(word1), len(word2)
        dp = [[0] * (n + 1) for _ in range(m + 1)]

        for i in range(m + 1):
            dp[i][0] = i
        for j in range(n + 1):
            dp[0][j] = j

        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if word1[i - 1] == word2[j - 1]:
                    dp[i][j] = dp[i - 1][j - 1]
                else:
                    dp[i][j] = min(dp[i - 1][j - 1], dp[i - 1][j], dp[i][j - 1]) + 1

        return dp[m][n]

# Example usage
trie = Trie()

for word in words:
    trie.insert(word)



In [22]:
# Get closest word
closest_word = trie.get_closest_word('pleas')
print("Closest word:", closest_word)  # 'apple'


Closest word: plead
